### 집계와 분류
대용량 데이터에 대해 기본적인 특성은 sum, mean, median, min, max와 같은 집계 연산을 수행한다. 이번 절에서는 numpy와 유사한 연산부터 groupby를 기반으로 하는 좀더 복잡한 연산까지 Pandas에서 제공하는 집계 연산을 살펴본다.

### 행성 데이터
이번에는 Seaborn 패키지를 통해 사용할 수 있는 행성 데이터 세트를 사용하겠다. 이 데이터는 천문학자가 다른 별 주변에서 발견한 행성에 대한 정보를 제공한다.

In [5]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [6]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [7]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


### Pandas의 간단한 집계 연산
집계 연산은 하나의 값을 반환한다.

In [8]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [9]:
ser.sum()

2.811925491708157

In [10]:
ser.mean()

0.5623850983416314

DataFrame의 경우 집계 함수는 기본적으로 각 열 내의 결과를 반환한다.

In [13]:
df = pd.DataFrame({'A': rng.rand(5),'B': rng.rand(5)})
df

,A,B
0,0.183405,0.611853
1,0.304242,0.139494
2,0.524756,0.292145
3,0.431945,0.366362
4,0.291229,0.456070


In [14]:
df.mean()

A    0.347115
B    0.373185
dtype: float64

In [15]:
df.sum()

A    1.735577
B    1.865923
dtype: float64

In [16]:
#axis 인수를 지정하면 각 행에 대해 집계할 수 있다.
df.mean(axis='columns')

0    0.397629
1    0.221868
2    0.408451
3    0.399153
4    0.373650
dtype: float64

In [17]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


Pandas에서 제공하는 집계 연산을 요약

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | 항목전체 개수           |
| ``first()``, ``last()``  | 첫 항목과 마지막 항목             |
| ``mean()``, ``median()`` | 평균값과 중앙값                 |
| ``min()``, ``max()``     | 최소값과 최대값             |
| ``std()``, ``var()``     | 표준편차와 분산 |
| ``mad()``                | 절대 평균 편차         |
| ``prod()``               | 전체 항목의 곱            |
| ``sum()``                | 전체 항목의 합                |

이것들으 모두 데이터프레임과 시리즈 객체에서 제공하는 매서드다

### GroupBy: 분할, 적용, 결합
간다한 집계는 데이터세트의 전반적인 특성을 알려주지만 어떤 레이블이나 인덱스를 기준으로 조건부로 집계하고 싶을 수 있다.
### 분할, 적용 결합
![](./../figures/03.08-split-apply-combine.png)
* 분할 : 지정된 키 값을 기준으로 DataFrame을 나누고 분류하는 단계다.
* 적용 : 개별 그룹 내에서 일반적으로 집계 변환, 필터링 같은 함수를 계산한다.
* 결합 : 이 연산의 결과를 결과 배열에 병합한다.

물론 이 작업을 엪에서 다룬 마스킹, 집계, 병합 명령어의 조합을 사용해 직접 수행할 수 있지만 중간 단계의 분할을 명시적으로 설명할 필요가 없다는 사실을 깨닫는 것이 중요하다. Groupby는 이 단계들을 추상화 하는데 있다. 사용자는 내부에서 어떻게 수행되는지 신경 쓸 필요 없이 전체 차원에서의 연산만 생각하면 된다.

In [18]:
df = pd.DataFrame({'key':['A','B','C','A','B','C'],
                  'data':range(6)}, columns=['key','data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [19]:
df.groupby('key')

데이터프레임의 groupby()매서드에 원하는 키 열의 이름을 전달해 가장 기본적인 분할-적용-결합 연산을 계산할 수 있다. 여기서 데이터프레임의 집합이 아니라 groupby 객체가 반환된다는데 주목하자. 이 객체는 그룹을 세부적으로 조사할 준비는 되어 있으나 집계 로직이 적용되기 까지는 아무 계산을 수행하지 않는 데이터프레임의 뷰로 생가갛면 된다. 이 게으른 평가(lazy evaluation) 바잇ㄱ은 일반 집계 연산이 사용자에게 거의 투명한 방식으로 매우 효율적으로 구현될 수 있다.

In [21]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


### GroupBy 객체
GroupBy 객체는 매우 유연한 추상화다. 여러 면에서 이 객체는 단순히 DataFrame 컬렉션처럼 취급할 수 있으며 내부적으로 어려운 일들을 처리한다. 행성 데이터를 사용해서 예제를 살펴보자. GroupBy에서 사용할 수 있는 가장 중요한 연산은 집계, 필터, 변환, 적용이다.  
열 인덱싱 GroupBy 객체는 DataFrame과 동일한 방식으로 열 인덱싱을 지원하며 수정된 GorupBy객체를 반환한다. 예를 들면 다음과 같다.

In [22]:
print(planets.groupby('method'))
print(planets.groupby('method')['orbital_period'])

In [23]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

각 그룹내 반복 GroupBy 객체는 그룹을 직접 순회할 수 있도록 지원하며, 각 그룹을 Series나 DataFrame으로 반환한다.

In [24]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


사실 내장된 apply 기능을 사용하는것이 대체로 더 빠르다. 디시패치 메서드(Dispatch Method) GorupBy 객체가 명시적으로 구현하지 않은 매서드는 그거시 DataFrame 객체든 Series 객체든 상관없이 일부 파이썬 클래스 매직을 통해 그 그룹에 전다로디고 호출될 것이다.

In [25]:
planets.groupby('method')['year'].describe().unstack()

       method                       
count  Astrometry                          2.0
       Eclipse Timing Variations           9.0
       Imaging                            38.0
       Microlensing                       23.0
       Orbital Brightness Modulation       3.0
                                         ...  
max    Pulsar Timing                    2011.0
       Pulsation Timing Variations      2007.0
       Radial Velocity                  2014.0
       Transit                          2014.0
       Transit Timing Variations        2014.0
Length: 80, dtype: float64

### 집계, 필터, 변환, 적용
GroupBy 객체에는 그룹 데이터를 결합하기 전에 여러 유용한 연산을 효율적으로 구현하는 aggregate(), filter(), transform(), apply() 메서드가 있다.

In [26]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


aggregate() 메서드는 훨씬 더 많은 유연성을 제공한다. 이 메서드는 문자열, 함수, 리스트 등을 취해 한 번에 모든 집계를 계산할 수 있다.

In [27]:
df.groupby('key').aggregate(['min', np.median, max]) // 문자열-함수-리스트

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [28]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


필터링 연산을 사용하면 그룹 속성을 기준으로 데이터를 걸러낼 수 있다. 예를 들어 표준 편차가 어떤 임계 값보다 큰 그룹을 모두 유지할 수 있다

In [29]:
def filter_func(x):
    return x['data2'].std() > 4

display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

filter()함수는 그룹이 필터링을 통과하는지 아닌지를 지정하는 부울 값을 반환한다. 여기서는 그룹A의 표준편차가 4보다 작으므로 결과에서 그 그룹이 제거된다.

변환: 집계는 데이터의 축소 버전을 반환해야 하지만. 변환은 재결합을 위해 전체 데이터의 변환된 버전을 반환할 수 있다. 그러한 변환의 경우 결과는 입력과 같은 형상을 가진다. 일반적인 예로 데이터에서 그룹별 평균값을 빼서 데이터를 중앙에 정렬하는 것을 들 수 있다.  
apply() 매서드는 임의의 함수를 그룹 결과에 적용할 때 사용한다. 이 함수는 DataFrame을 취해 Pandas 객체(즉, DataFrame, Series)나 스칼라를 반환한다. 결합 연산은 반환된 출력값 유형에 따라 조정된다.

In [30]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3


GroupBy내에서 apply()는 상당히 유연하다. 함수는 DaaFrame을 취하고 Pandas객체나 스칼라는 반환한다는 것이 유일한 규칙이다. 그 중간에 무엇을 하든지 상관없다.

### 분할 키 지정하기
앞에서 소개한 간단한 예제에서 하나의 열 이름을 기준으로 DataFrame을 분할했다. 이것은 그룹을 정의하는 여러 방식중 하나이고 이제부터 그룹을 지정하는 다른 방식을 살펴보자

In [31]:
L = [0,1,0,1,2,0]
print(df); print(df.groupby(L).sum())

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9
   data1  data2
0      7     17
1      4      3
2      4      7


이전의 df.groupby('key)를 구현하는 더 자세한 방식이 있다.

In [32]:
display('df', "df.groupby(df['key']).sum()")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(df['key']).sum()
     data1  data2
key              
A        3      8
B        5      7
C        7     12

인덱스를 그룹에 매핑한 딕셔너리나 시리즈 또 다른 방법은 인덱스 값을 그룹 키에 매핑하는 딕셔너리를 제공하는 것이다.

In [33]:
df2 = df.set_index('key')
mapping = {'A':'vowel','B':'consonant','C':'consonant'}
print(df2); print(df2.groupby(mapping).sum())

     data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9
           data1  data2
key                    
consonant     12     19
vowel          3      8


매핑과 유사하게 인덱스 값을 ㅣㅂ력해서 그룹을 출력하는 파이썬 함수를 전달하면 된다.

In [34]:
print(df2); print(df2.groupby(str.lower).mean())

     data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9
     data1  data2
key              
a      1.5    4.0
b      2.5    3.5
c      3.5    6.0


In [35]:
df2.groupby([str.lower,mapping]).mean()

,,data1,data2
key,key,,
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0


### 분류(Grouping) 예제

In [36]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


### 피벗 테이블
피벗 테이블은 표 형태의 데이터로 작업하는 스프레드시트와 다른 프로그램에서 일반적으로 볼 수 있는 유사한 작업이다. 피벗 테이블은 입력값으로 간단한 열 단위의 데이터를 취하고 그 데이터에 대한 다차원 요약을 제공하는 2차원 테이블로 항목을 그룹핑한다.  

피벗 테이블을 근본적으로 GroupBy집계의 다차원 버전이라고 생각하면 도움이 된다. 다시 말해 분할-적용-결합 작업을 하면 분할과 결합 작업이 1차원 인덱스에서 발생하는 것이 아니라 1차원 그리드에서 발생한다.

In [54]:
import numpy as np
import pandas as pd
import seaborn as sns
titanic = sns.load_dataset('titanic')

### 왜 피벗 테이블이 필요한가?

In [55]:
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


In [56]:
titanic.groupby(['sex','class'])['survived'].aggregate('mean').unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


groupby와 aggregate로 표현을 할 수 있지만 원하는 정보가 늘어날 수록 코드가 지저분해지기 시작한다.

### 피벗 테이블 구문
pivot_table 매서드를 사용해서 동일하게 구현해보자

In [57]:
titanic.pivot_table('survived',index='sex', columns='class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


앞선 코드보다 읽기 쉬우면서도 똑같은 결과를 만들어낸다.

### 다단계 피벗 테이블
200p